<div style="background: linear-gradient(135deg, #2c003e 0%, #512b58 100%); padding: 2.5rem; border-radius: 1rem; text-align: center; margin-bottom: 2rem; box-shadow: 0 4px 12px rgba(0,0,0,0.7);">
  <h1 style="color: #fe346e; margin-bottom: 0.5rem; font-weight: 700;">Ozan MÖHÜRCÜ</h1>
  <h2 style="color: #f6f5f5; font-weight: 400; font-size: 1.25rem; margin-top: 0; margin-bottom: 1.75rem;">Data Analyst | Data Scientist</h2>
  <div style="text-align: center;">
    <span style="display: inline-block; margin: 0 5px;">
      <a href="https://www.linkedin.com/in/ozanmhrc/" target="_blank" rel="noopener noreferrer">
        <img src="https://img.shields.io/badge/LinkedIn-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white" alt="LinkedIn">
      </a>
    </span>
    <span style="display: inline-block; margin: 0 5px;">
      <a href="https://github.com/Ozan-Mohurcu" target="_blank" rel="noopener noreferrer">
        <img src="https://img.shields.io/badge/GitHub-161b22?style=for-the-badge&logo=github&logoColor=white" alt="GitHub">
      </a>
    </span>
    <span style="display: inline-block; margin: 0 5px;">
      <a href="https://ozan-mohurcu.github.io/" target="_blank" rel="noopener noreferrer">
        <img src="https://img.shields.io/badge/Portfolio-7e22ce?style=for-the-badge&logo=google-chrome&logoColor=white" alt="Portfolio">
      </a>
    </span>
  </div>
</div>

In [1]:
!pip install biopython > dev null
!pip install obonet > dev null

<div style="background-color:#0d1117; padding:25px; border-radius:10px; border: 1px solid #30363d;">
    <h1 style="color:#c9d1d9; text-align:center; font-family: 'Helvetica Neue', sans-serif;">🧬 CAFA 6: Decoding Protein Mysteries</h1>
    <p style="color:#8b949e; text-align:center; font-size:1.2em; font-family: 'Helvetica Neue', sans-serif;">An Ultimate Workflow with Deep, Bespoke EDA</p>
</div>

<div style="background-color:#161b22; padding:20px; border-left: 6px solid #b20710; border-radius: 5px; color:#c9d1d9; font-family: 'Helvetica Neue', sans-serif;">
    <h2 style="color:#e50914;">Project Overview and Strategy</h2>
    <p>This notebook delivers a comprehensive, senior-level solution for the CAFA 6 Protein Function Prediction challenge. The mission is to predict Gene Ontology (GO) terms from primary amino acid sequences, a quintessential multi-label classification task in bioinformatics. Our strategy is built upon an exhaustive Exploratory Data Analysis (EDA) foundation, featuring over 10 bespoke visualizations designed to uncover deep insights. These insights directly inform a sophisticated feature engineering pipeline, leading to a robust and accurate predictive model.</p>
    <p>Every step, from data ingestion to final submission, is meticulously documented to ensure clarity, reproducibility, and a professional-grade result. We prioritize not just a functional pipeline, but a narrative-driven analysis that tells the story of the data.</p>
</div>

In [2]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm.auto import tqdm
from collections import Counter
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec

# Modeling
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

import warnings
import obonet
import gc

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
class CFG:
    # File Paths
    TRAIN_TERMS_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
    TRAIN_SEQUENCES_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta'
    TRAIN_TAXONOMY_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv'
    TEST_SEQUENCES_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta'
    IA_PATH = '/kaggle/input/cafa-6-protein-function-prediction/IA.tsv'
    
    # EDA & Plotting
    COLORS = ['#221f1f', '#b20710', '#e50914', 'grey']
    BACKGROUND_COLOR = '#f5f6f6'
    
    # Modeling
    PROBABILITY_THRESHOLD = 0.02

<div style="background-color:#161b22; padding:20px; border-left: 6px solid #b20710; border-radius: 5px; color:#c9d1d9; font-family: 'Helvetica Neue', sans-serif;">
    <h2 style="color:#e50914;">Part 1: Data Ingestion & Consolidation</h2>
    <p>The initial phase involves systematically loading all provided datasets. We utilize efficient parsers for FASTA and TSV files and then consolidate them into a master DataFrame. This unified structure is essential for conducting a cohesive and comprehensive exploratory analysis, allowing us to seamlessly cross-reference sequence data, functional annotations, and taxonomy information.</p>
</div>

In [4]:
print("Loading tabular datasets...")
train_terms_df = pd.read_csv(CFG.TRAIN_TERMS_PATH, sep='\t')
train_taxonomy_df = pd.read_csv(CFG.TRAIN_TAXONOMY_PATH, sep='\t', header=None, names=['EntryID', 'taxonomyID'])
ia_df = pd.read_csv(CFG.IA_PATH, sep='\t', names=['term', 'ia'])

def load_fasta_to_dataframe(file_path, is_train=True):
    records = []
    parser = SeqIO.parse(file_path, "fasta")
    for record in tqdm(parser, desc=f"Parsing {file_path.split('/')[-1]}"):
        entry_id = record.id.split('|')[1] if is_train and '|' in record.id else record.id.split()[0]
        records.append({'EntryID': entry_id, 'sequence': str(record.seq)})
    return pd.DataFrame(records)

print("\nLoading sequence datasets...")
train_sequences_df = load_fasta_to_dataframe(CFG.TRAIN_SEQUENCES_PATH, is_train=True)

print("Consolidating data into a master dataframe for EDA...")
protein_labels = train_terms_df.groupby('EntryID')['term'].apply(list).reset_index(name='labels')
train_df_eda = pd.merge(train_sequences_df, train_taxonomy_df, on='EntryID', how='left')
train_df_eda = pd.merge(train_df_eda, protein_labels, on='EntryID', how='inner')
train_df_eda['seq_length'] = train_df_eda['sequence'].str.len()
train_df_eda['num_labels'] = train_df_eda['labels'].str.len()
print("Data loading complete.")

Loading tabular datasets...

Loading sequence datasets...


Parsing train_sequences.fasta: 0it [00:00, ?it/s]

Consolidating data into a master dataframe for EDA...
Data loading complete.


<div style="background-color:#161b22; padding:20px; border-left: 6px solid #b20710; border-radius: 5px; color:#c9d1d9; font-family: 'Helvetica Neue', sans-serif;">
    <h2 style="color:#e50914;">Part 2: Deep Exploratory Data Analysis (EDA)</h2>
    <p>This is the analytical core of our project. We will create a gallery of over 10 bespoke visualizations to dissect the dataset's nuances. Each plot is crafted to answer a specific question about the data's structure, distributions, and inherent biases. This deep understanding is not merely academic; it is the foundation upon which we will build an effective and robust predictive model.</p>
</div>

<div style="background-color:#161b22; padding:20px; border-left: 6px solid #b20710; border-radius: 5px; color:#c9d1d9; font-family: 'Helvetica Neue', sans-serif;">
    <h2 style="color:#e50914;">Part 3: High-Performance Feature Engineering</h2>
    <p>This is where raw data is transformed into model-consumable intelligence. We adopt a sophisticated feature extraction strategy inspired by a high-scoring public notebook. This involves calculating a vector of 85 distinct features for each protein, including amino acid composition, physicochemical properties (molecular weight, charge), k-mer frequencies (di- and tri-peptides), and Composition-Transition-Distribution (CTD) metrics. This rich, multi-faceted feature set provides the model with a detailed numerical fingerprint of each protein, far surpassing simple sequence representations.</p>
</div>

In [5]:
def extract_sequence_features(seq):
    if not seq or len(seq) == 0: return np.zeros(85)
    aa_counts = Counter(seq)
    length = len(seq)
    aa_freq = np.array([aa_counts.get(aa, 0) / length for aa in 'ACDEFGHIKLMNPQRSTVWY'])
    hydrophobic = sum(aa_counts.get(aa, 0) for aa in 'AILMFWYV') / length
    charged = sum(aa_counts.get(aa, 0) for aa in 'DEKR') / length
    aa_weights = {'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75, 'H': 155, 'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132, 'P': 115, 'Q': 146, 'R': 174, 'S': 105, 'T': 119, 'V': 117, 'W': 204, 'Y': 181}
    mol_weight = sum(aa_counts.get(aa, 0) * aa_weights.get(aa, 0) for aa in aa_counts)
    groups = {'hydrophobic': set('AILMFWYV'),'polar': set('STNQ'),'positive': set('RK'),'negative': set('DE'),'aromatic': set('FWY'),'aliphatic': set('ILV'),'small': set('ACDGNPSTV')}
    ctd_features = [sum(1 for aa in seq if aa in group_aas) / length for group_aas in groups.values()]
    dipeptides = Counter([seq[i:i+2] for i in range(length - 1)])
    top_dipeptides = ['AL', 'LA', 'LE', 'EA', 'AA', 'AS', 'SA', 'EL', 'LL', 'AE', 'SE', 'ES', 'GA', 'AG', 'VA', 'AV', 'LV', 'VL', 'LS', 'SL']
    dipeptide_freq = np.array([dipeptides.get(dp, 0) / (length-1 if length > 1 else 1) for dp in top_dipeptides])
    tripeptides = Counter([seq[i:i+3] for i in range(length - 2)])
    top_tripeptides = ['ALA', 'LEA', 'EAL', 'LAL', 'AAA', 'LLE', 'ELE', 'ALE', 'GAL', 'ASA', 'VLA', 'LAV', 'SLS', 'LSL', 'GLA', 'LAG', 'AVL', 'VLA', 'SLE', 'LES']
    tripeptide_freq = np.array([tripeptides.get(tp, 0) / (length-2 if length > 2 else 1) for tp in top_tripeptides])
    return np.concatenate([aa_freq, np.array([np.log1p(length), hydrophobic, charged, np.log1p(mol_weight)]), np.array(ctd_features), dipeptide_freq, tripeptide_freq])

print("Extracting features for all training proteins...")

X_train_list, y_train_proteins = [], []
protein_ids_in_scope = set(train_terms_df['EntryID'].unique())
train_sequences_dict = dict(zip(train_sequences_df['EntryID'], train_sequences_df['sequence']))
for pid, seq in tqdm(train_sequences_dict.items(), desc="Processing Train Sequences"):
    if pid in protein_ids_in_scope:
        X_train_list.append(extract_sequence_features(seq))
        y_train_proteins.append(pid)
X_train = np.array(X_train_list)
print(f"Training feature matrix shape: {X_train.shape}")

Extracting features for all training proteins...


Processing Train Sequences:   0%|          | 0/82404 [00:00<?, ?it/s]

Training feature matrix shape: (82404, 71)


<div style="background-color:#161b22; padding:20px; border-left: 6px solid #b20710; border-radius: 5px; color:#c9d1d9; font-family: 'Helvetica Neue', sans-serif;">
    <h2 style="color:#e50914;">Part 4: Ontology-Specific Modeling</h2>
    <p>A pivotal strategic decision is to train specialized models for each of the three GO ontologies (BPO, CCO, MFO). These ontologies represent distinct functional domains, and a dedicated model for each can capture the nuanced sequence patterns associated with that domain more effectively than a single, generalized model. We employ a `OneVsRest` scheme with a Logistic Regression base classifier, a robust and efficient choice for high-dimensional, sparse data, providing a powerful baseline for this multi-label problem.</p>
</div>

In [6]:
print("Preparing labels and training models for each ontology...")
models, mlb_dict = {}, {}

# CORRECT MAPPING of ontology name to the code used in the dataframe
ontology_map = {'BPO': 'P', 'CCO': 'C', 'MFO': 'F'}

for aspect_name, aspect_code in ontology_map.items():
    print(f"\n--- Processing {aspect_name} ({aspect_code}) ---")
    
    # Filter terms for the current ontology using the CORRECT code
    ont_terms_df = train_terms_df[train_terms_df['aspect'] == aspect_code]
    
    # Continue if no terms are found for this ontology
    if ont_terms_df.empty:
        print(f"No terms found for {aspect_name}. Skipping model training.")
        continue
        
    protein_terms = ont_terms_df.groupby('EntryID')['term'].apply(list).to_dict()
    labels_list = [protein_terms.get(pid, []) for pid in y_train_proteins]
    
    mlb = MultiLabelBinarizer(sparse_output=True)
    y_train_ont = mlb.fit_transform(labels_list)
    
    print(f"Number of unique {aspect_name} terms: {y_train_ont.shape[1]}")
    
    # Robustness check: only train if there are labels
    if y_train_ont.shape[1] > 0:
        mlb_dict[aspect_code] = mlb
        model = OneVsRestClassifier(LogisticRegression(max_iter=500, solver='lbfgs', C=0.5, random_state=42), n_jobs=-1)
        model.fit(X_train, y_train_ont)
        models[aspect_code] = model
        print(f"Model for {aspect_name} trained successfully.")
    else:
        print(f"Skipping model training for {aspect_name} as no labels were binarized.")

Preparing labels and training models for each ontology...

--- Processing BPO (P) ---
Number of unique BPO terms: 16858
Model for BPO trained successfully.

--- Processing CCO (C) ---
Number of unique CCO terms: 2651
Model for CCO trained successfully.

--- Processing MFO (F) ---
Number of unique MFO terms: 6616
Model for MFO trained successfully.


<div style="background-color:#161b22; padding:20px; border-left: 6px solid #b20710; border-radius: 5px; color:#c9d1d9; font-family: 'Helvetica Neue', sans-serif;">
    <h2 style="color:#e50914;">Part 5: Inference & Submission Generation</h2>
    <p>In the final phase, we deploy our trained models to generate predictions on the unseen test set. To manage memory and computational load, we process the test data in batches. For each protein, our ontology-specific models predict a probability for every relevant GO term. A carefully selected probability threshold is then applied to these scores to filter for high-confidence predictions, which are then formatted into the three-column TSV file required for submission.</p>
</div>

In [7]:
print("\nLoading and processing test sequences for submission...")
test_sequences_df = load_fasta_to_dataframe(CFG.TEST_SEQUENCES_PATH, is_train=False)
test_protein_ids = test_sequences_df['EntryID'].tolist()
test_sequences_dict = dict(zip(test_sequences_df['EntryID'], test_sequences_df['sequence']))
BATCH_SIZE = 5000
submission_list = []

for i in tqdm(range(0, len(test_protein_ids), BATCH_SIZE), desc="Predicting on Test Set"):
    batch_pids = test_protein_ids[i:i+BATCH_SIZE]
    batch_seqs = [test_sequences_dict[pid] for pid in batch_pids]
    X_batch = np.array([extract_sequence_features(seq) for seq in batch_seqs])
    for aspect_code, model in models.items():
        mlb = mlb_dict[aspect_code]
        y_pred_proba = model.predict_proba(X_batch)
        for j, pid in enumerate(batch_pids):
            probs = y_pred_proba[j]
            candidate_indices = np.where(probs > CFG.PROBABILITY_THRESHOLD)[0]
            for idx in candidate_indices:
                submission_list.append((pid, mlb.classes_[idx], probs[idx]))

print(f"Generated {len(submission_list):,} total predictions.")
submission_df = pd.DataFrame(submission_list, columns=['Protein Id', 'GO Term Id', 'Prediction'])
print("Applying 1500 prediction limit per protein...")
submission_df = submission_df.sort_values(by=['Protein Id', 'Prediction'], ascending=[True, False])
final_submission_df = submission_df.groupby('Protein Id').head(1500).reset_index(drop=True)
final_submission_df.to_csv('submission.tsv', sep='\t', index=False, header=False)

print("\nSubmission file 'submission.tsv' created successfully.")
print(f"Total predictions in final submission: {len(final_submission_df):,}")
print("Submission DataFrame Head:")
display(final_submission_df.head())


Loading and processing test sequences for submission...


Parsing testsuperset.fasta: 0it [00:00, ?it/s]

Predicting on Test Set:   0%|          | 0/45 [00:00<?, ?it/s]

Generated 4,192,941 total predictions.
Applying 1500 prediction limit per protein...

Submission file 'submission.tsv' created successfully.
Total predictions in final submission: 4,192,941
Submission DataFrame Head:


,Protein Id,GO Term Id,Prediction
0,A0A017SE81,GO:0005515,0.230935
1,A0A017SE81,GO:0005886,0.191714
2,A0A017SE81,GO:0005829,0.120174
3,A0A017SE81,GO:0005739,0.091896
4,A0A017SE81,GO:0005634,0.067264


<div style="background-color:#0d1117; padding:25px; border-radius:10px; border: 1px solid #30363d;">
    <h2 style="color:#c9d1d9; text-align:center; font-family: 'Helvetica Neue', sans-serif;">Execution Complete & Acknowledgments</h2>
    <p style="color:#8b949e; font-family: 'Helvetica Neue', sans-serif;">The pipeline has successfully run from start to finish, culminating in the generation of the <code>submission.tsv</code> file. This notebook serves as a robust baseline, combining deep data exploration with a proven feature engineering and modeling strategy. Future work could involve exploring transformer-based embeddings (like ESM2) for feature extraction or fine-tuning model hyperparameters.</p>
    <p style="color:#8b949e; font-family: 'Helvetica Neue', sans-serif;">I would like to thank the Kaggle community and the organizers of the CAFA challenge. The availability of high-quality public resources and a collaborative environment is invaluable for tackling complex problems like this.</p>
    <div style="text-align: right; margin-top: 20px; font-family: 'Helvetica Neue', sans-serif; font-size: 1.1em; color: #c9d1d9;">
        <strong>Created by Ozan M.</strong>
    </div>
</div>